In [5]:
#importing important libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from skmultilearn.model_selection import iterative_train_test_split
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss,f1_score,confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import tensorflow_addons as tfa
import pickle
from tensorflow.keras import backend

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
#reading train dataset

train_features=pd.read_csv('../input/lish-moa/train_features.csv')
test_features=pd.read_csv('../input/lish-moa/test_features.csv')
train_targets_scored=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored=pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug=pd.read_csv('../input/lish-moa/train_drug.csv')

In [5]:
import random
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(seed=42)

In [6]:
n_folds=10
SEEDS=[660,42,2020,72,64]

In [7]:
#list of names of the MoAs in the dataset
target_cols_scored=[col for col in train_targets_scored.columns if col not in ['sig_id']]
target_cols_nonscored=[col for col in train_targets_nonscored.columns if col not in ['sig_id']]

In [8]:
#separating gene feautres and cell features in a separate list
#we know that all the gene expression features name starts with 'g-' and cell features name starts with 'c-', hence trying to observe set of all the gene expression as one function and all the cell functions as one features
gene_features=[]
cell_features=[]
for i in train_features.columns:
    if i.startswith('g-'):
        gene_features.append(i)
    if i.startswith('c-'):
        cell_features.append(i)

In [9]:
#merging the train dataset and train target scored dataset into one
train_merge=pd.merge(train_features,train_targets_scored,on="sig_id",how="left")

In [10]:
train_merge=train_merge[train_merge['cp_type']!='ctl_vehicle']
#test_features1=test_features.copy()
test_features1=test_features[test_features['cp_type']!='ctl_vehicle']

In [11]:
train_merge.drop(columns=['cp_type'],inplace=True)
test_features1.drop(columns=['cp_type'],inplace=True)

In [12]:
lb=LabelEncoder()
lb.fit(train_merge['cp_time'])

train_merge['cp_time']=lb.transform(train_merge['cp_time'])
test_features1['cp_time']=lb.transform(test_features1['cp_time'])
##################################################################################
lb1=LabelEncoder()
lb1.fit(train_merge['cp_dose'])

train_merge['cp_dose']=lb1.transform(train_merge['cp_dose'])
test_features1['cp_dose']=lb1.transform(test_features1['cp_dose'])

In [13]:
feature_groups=gene_features+cell_features+['cp_time','cp_dose']


In [14]:
train_dataset=train_merge[feature_groups]


In [15]:
test_dataset=test_features1[feature_groups]


In [16]:
feature_value=train_dataset.iloc[15]
Y=train_merge[target_cols_scored].iloc[15]


In [17]:
def function1(feature_value):
    '''
    The function returns the MoA predictions given then feature variables
    
    '''
    nfolds=10
    SEEDS=[660,42,2020,72,64]
    predicted_value=np.zeros(())
    y_pred_model1_seed_level=np.zeros((1,206))
    y_pred_model2_seed_level=np.zeros((1,206))
    y_pred_model3_seed_level=np.zeros((1,206))
    y_pred_model4_seed_level=np.zeros((1,206))
    for seed in SEEDS:
        seed_everything(seed)
        y_pred_model1_fold_level=np.zeros((1,206))
        y_pred_model2_fold_level=np.zeros((1,206))
        y_pred_model3_fold_level=np.zeros((1,206))
        y_pred_model4_fold_level=np.zeros((1,206))
        for fold in range(nfolds):
            feature_value1=feature_value.copy()
            feature_value2=feature_value.copy()
            pca_gene1_3_4=pickle.load(open(f'../input/inference-version24/pickle_folder_gene/pca_gene_model1_3_4_seed{seed}_fold{fold}.pkl','rb'))
            pca_cell1_3_4=pickle.load(open(f'../input/inference-version24/pickle_folder_cell/pca_cell_model1_3_4_seed{seed}_fold{fold}.pkl','rb'))
            qt1_3_4=pickle.load(open(f'../input/inference-version24/pickle_folder_qt/qt_model_1_3_4_seed{seed}_fold{fold}.pkl','rb'))
            
            pca_gene_2=pickle.load(open(f'../input/inference-version24/pickle_folder_gene/pca_gene_model2_seed{seed}_fold{fold}.pkl','rb'))
            pca_cell_2=pickle.load(open(f'../input/inference-version24/pickle_folder_cell/pca_cell_model2_seed{seed}_fold{fold}.pkl','rb'))
            qt_2=pickle.load(open(f'../input/inference-version24/pickle_folder_qt/qt_model2_seed{seed}_fold{fold}.pkl','rb'))
            
            pca_gene_features=pca_gene1_3_4.transform(feature_value2[gene_features].values.reshape(1,-1))
            pca_cell_features=pca_cell1_3_4.transform(feature_value2[cell_features].values.reshape(1,-1))
     
            feature_value2=pd.concat([pd.DataFrame(feature_value2.values.reshape(1,-1)),pd.DataFrame(pca_gene_features),pd.DataFrame(pca_cell_features)],axis=1)
            
            feature_value2=qt1_3_4.transform(feature_value2)
            
            
            
            #MODEL-1------------------------------------------------------------------------------------------------------------------------------
            model1=load_model(f'../input/model-1-simple-nn-for-ensembling-ver1/model1_simple_nn_blending_seed{seed}_fold{fold}.h5')  
            y_pred1=model1.predict(feature_value2)
            
            #MODEL-2------------------------------------------------------------------------------------------------------------------------------
            
            model2=load_model(f'../input/model-2-cnn-for-ensembling-ver1/model_cnn_seed{seed}_fold{fold}.h5')
            pca_gene_features=pca_gene_2.transform(feature_value1[gene_features].values.reshape(1,-1))
            pca_cell_features=pca_cell_2.transform(feature_value1[cell_features].values.reshape(1,-1))
            feature_value1=pd.concat([pd.DataFrame(feature_value1.values.reshape(1,-1)),pd.DataFrame(pca_gene_features),pd.DataFrame(pca_cell_features)],axis=1)
            feature_value1=qt_2.transform(feature_value1)
            y_pred2=model2.predict(feature_value1)
            
            #MODEL-3-------------------------------------------------------------------------------------------------------------------------------
            model3=load_model(f'../input/model-3-simple-rnn-ensembling-ver1/model3_simple_nn_blending_seed{seed}_fold{fold}.h5')
            y_pred3=model3.predict(feature_value2)
            
            
            #MODEL-4--------------------------------------------------------------------------------------------------------------------------------
            
            model4=load_model(f'../input/model-4-simple-rnn-ensembling-ver1/model4_simple_nn_blending_seed{seed}_fold{fold}.h5')
            y_pred4=model4.predict(feature_value2)
            
         
            ########################################################################################################################################
            y_pred_model1_fold_level=y_pred_model1_seed_level+y_pred1
            y_pred_model2_fold_level=y_pred_model2_seed_level+y_pred2
            y_pred_model3_fold_level=y_pred_model3_seed_level+y_pred3
            y_pred_model4_fold_level=y_pred_model4_seed_level+y_pred4
       
        y_pred_model1_seed_level=y_pred_model1_seed_level+y_pred_model1_fold_level
        y_pred_model2_seed_level=y_pred_model2_seed_level+y_pred_model2_fold_level
        y_pred_model3_seed_level=y_pred_model3_seed_level+y_pred_model3_fold_level
        y_pred_model4_seed_level=y_pred_model4_seed_level+y_pred_model4_fold_level
        
        y_pred_model1_seed_level=y_pred_model1_seed_level/(nfolds*len(SEEDS))
        y_pred_model2_seed_level=y_pred_model2_seed_level/(nfolds*len(SEEDS))
        y_pred_model3_seed_level=y_pred_model3_seed_level/(nfolds*len(SEEDS))
        y_pred_model4_seed_level=y_pred_model4_seed_level/(nfolds*len(SEEDS))
        
    y_pred_final=(0.85)*y_pred_model1_seed_level+(0.08)*y_pred_model2_seed_level+(0.02)*y_pred_model3_seed_level+(0.05)*y_pred_model4_seed_level
    return y_pred_final
        
            
     

In [19]:
y_pred_final=function1(feature_value)

In [21]:
y_pred_final

(1, 206)

In [22]:
feature_value=train_dataset.iloc[15]
Y=train_merge[target_cols_scored].iloc[15]

In [23]:
y_pred_final.shape

(1, 206)

In [24]:
def function2(feature_value,Y):
    '''
    The function returns the logloss given the feature variable and truth values of MoA predictions.
    '''
    y_true=Y.values
    y_pred=function1(feature_value)
    return f'The loss value is {-backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))}'

In [25]:
function2(feature_value,Y)


'The loss value is 0.043939017812710875'